In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from typing import Any, Literal

import cv2
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter

from model.depth import convert_video_to_depth
from model.dmd import (
    ROOT,
    convert_depth_images_to_video,  # noqa F401
    convert_frames_to_video,
    extract_frames,
)

plt.rcParams['font.size'] = 16


In [ ]:
session = [
    'gA_1_s1_2019-03-08T09;31;15+01;00',
    'gA_1_s2_2019-03-08T09;21;03+01;00',
    'gA_1_s3_2019-03-14T14;31;08+01;00',
][2]
source_type: Literal['rgb', 'depth'] = 'depth'
source_video_extension: Literal['mp4', 'avi'] = (
    'avi' if source_type == 'depth' else 'mp4'
)
input_video_path = (
    ROOT / session / f'{session}_{source_type}_body.{source_video_extension}'
)
annotations_file_path = ROOT / session / f'{session}.json'

assert input_video_path.exists(), f'File not found: {input_video_path}'
assert annotations_file_path.exists(), f'File not found: {annotations_file_path}'
print(f'Input video path: {input_video_path}')
print(f'Annotations file path: {annotations_file_path}')

In [ ]:
# Extract frames from source video based on annotations
extract_frames(
    input_video_path=input_video_path,
    annotations_file_path=annotations_file_path,
    force_overwrite=False,
    source_type=source_type,
    skip_output_directory_setup=True,
)

In [ ]:
# RGB frames to video (also resizes to 518x518)
convert_frames_to_video(session, source_type=source_type, preset='slow', crf=10)

In [ ]:
# Video Depth Anything
convert_video_to_depth(ROOT / session, source_type='crop_rgb')

In [ ]:
# Visualization with a color map
# convert_depth_images_to_video(
#     session, 'anomal', 4, extension='png', fps=30, directory='video_depth_anything'
# )

In [ ]:
# import shutil

# all_dirs = sorted([p for p in (ROOT / session).rglob('depth_sensor') if p.is_dir()])
# for dir in all_dirs:
#     shutil.rmtree(dir)
#     print(f'Removed {dir}')

## Intel Realsense Depth Video

In [ ]:
cap = cv2.VideoCapture(str(input_video_path))
cap.set(cv2.CAP_PROP_CONVERT_RGB, 0)  # Disable RGB conversion
cap.set(cv2.CAP_PROP_POS_FRAMES, 653)
ret, frame = cap.read()
cap.release()

print(frame.min(), frame.max(), frame.dtype, frame.shape)

# Define the threshold (e.g. 2000 mm for 2 meters)
depth_threshold = 2000

# Clip the depth values so that any value above the threshold is set to the threshold
# img_clipped = np.clip(frame, 0, depth_threshold)
img_clipped = np.where(frame > depth_threshold, 0, frame)

# Map the range [0, depth_threshold] to [0, 255]
img8 = ((img_clipped / depth_threshold) * 255).astype(np.uint8)

plt.figure(figsize=(11, 6))
plt.imshow(img8, cmap='gray')
plt.axis('off')
plt.show()


def tick_formatter(x: Any, pos: Any) -> str:
    """Formatter function that scales tick values from 0-255 to 0-2000"""
    return f'{int(round(x * 2000 / 255))}'


plt.figure(figsize=(12, 7))
plt.imshow(img8, cmap=plt.cm.inferno)  # type: ignore

cbar = plt.colorbar(shrink=0.75)
cbar.ax.yaxis.set_major_formatter(FuncFormatter(tick_formatter))
cbar.set_label('Depth (mm)', rotation=270, labelpad=30)

plt.axis('off')
plt.show()

In [ ]:
extract_frames(
    input_video_path=input_video_path,
    annotations_file_path=annotations_file_path,
    force_overwrite=False,
    source_type=source_type,
    skip_output_directory_setup=True,
)

In [ ]:
convert_depth_images_to_video(
    session, 'normal', 1, extension='png', fps=30, directory='source_depth'
)